This Notebook downloads data from PC in the format required by the inference notebook.


In [1]:
import requests
import planetary_computer
from pystac.extensions.eo import EOExtension as eo
import pystac_client
import planetary_computer
import geopandas as gpd
from pathlib import Path
import numpy as np
import requests
import json
from datetime import datetime
import shapely
import pandas as pd
import rasterio as rio
from tqdm.auto import tqdm
import os
from shapely.geometry import Polygon, shape
from shapely.ops import transform
import pyproj
from rasterio.merge import merge

/var/folders/kq/rs7d6smx5kv8n5mp22x8kgjr0000gq/T/ipykernel_42888/3607905791.py:6: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopanda

In [2]:
# this is the world tide api key, let me know if you need it
planetary_computer_key_file = Path.cwd() / "planitary_computer_key.txt"
planetary_computer_api_key = planetary_computer_key_file.read_text().strip()
os.environ["PC_SDK_SUBSCRIPTION_KEY"] = planetary_computer_api_key

In [3]:
# this is the world tide api key, let me know if you need it
world_tide_key_file = Path.cwd() / "world_tide_key.txt"
world_tides_api_key = world_tide_key_file.read_text().strip()

In [4]:
# this is output dir, each scene will end up about 2Gb so make sure you have space!
export_dir = Path("/Volumes/4TB SSD/Coastline data") / "inference_scenes_3"
export_dir.mkdir(exist_ok=True, parents=True)
export_dir

PosixPath('/Volumes/4TB SSD/Coastline data/inference_scenes_3')

In [5]:
required_bands = ["B03", "B08"]
extract_start_year = 2022
extract_end_year = 2022

In [6]:
# this is the sentinel 2 grid, limited to just Tas at the moment
sentinel_2_grid = Path.cwd() / "data/Senntinele 2 grid coastal tas.gpkg"
s2_grid = gpd.read_file(sentinel_2_grid)

In [7]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
)

In [8]:
# sort the scenes by cloud cover
def sort_by_clouds(row, items):
    filtered_items = [
        item
        for item in items
        if eo.ext(item).cloud_cover is not None and row.Name in item.id
    ]
    sorted_items = sorted(filtered_items, key=lambda x: eo.ext(x).cloud_cover)
    return sorted_items

In [9]:
# use the world tide api to get the tide height at the time of the image
def filter_by_tide(centroid, items, limit=6):
    results = []
    for item in items:
        lon, lat = centroid.coords[0]

        dt_str = item.to_dict()["properties"]["datetime"]

        dt_obj = datetime.fromisoformat(dt_str.replace("Z", "+00:00"))

        # Fetch data from API
        url = f"https://www.worldtides.info/api/v3?heights&date={dt_obj.date().isoformat()}&lat={lat}&lon={lon}&key={world_tides_api_key}"
        response = requests.get(url)
        data = json.loads(response.text)

        if data == {"status": 400, "error": "No location found"}:
            print("No location found in world tides")
            return pd.DataFrame({"items": items, "tide": np.full(len(items), np.nan)})[
                :limit
            ]

        min_diff = float("inf")

        target_timestamp = dt_obj.timestamp()
        for entry in data["heights"]:
            diff = abs(entry["dt"] - target_timestamp)
            if diff < min_diff:
                min_diff = diff
                closest_entry = entry

        results.append({"item": item, "tide": closest_entry["height"]})
    # convert to df and sort by tide height
    results_df = pd.DataFrame(results).sort_values(by="tide", ascending=False)[:limit]

    return results_df

In [10]:
# sign the url and download the band
def get_band(href, attempt=0):
    try:
        singed_href = planetary_computer.sign(href)
        with rio.open(singed_href) as src:
            return src.read(1), src.profile.copy()
    except:
        print(f"Failed to open {href}")
        if attempt < 3:
            print(f"Trying again {attempt+1}")
            return get_band(href, attempt + 1)
        else:
            print(f"Failed to open {href} after 3 attempts")
            return None, None

In [11]:
# downlaod the required number of bands
def downlaod_bands(items_with_tide, time_steps):
    bands = []
    profile = {}
    pbar = tqdm(total=time_steps * len(required_bands), leave=False)
    for id, row in items_with_tide.iterrows():
        scene_bands = []

        for band in required_bands:
            href = row["item"].assets[band].href
            band, profile = get_band(href)
            if type(band) == type(None):
                print(f"Failed to download {href}")
                scene_bands = []
                break
            pbar.update(1)

            scene_bands.append(band)
        for band in scene_bands:
            bands.append(band)
        if len(bands) == time_steps * len(required_bands):
            return bands, profile
    return bands, profile

In [12]:
# each grid location may have multiple orbits which covers it, list each orbit
def split_by_orbits(items):
    orbits = {}
    for item in items:
        orbit = item.properties["sat:relative_orbit"]
        if orbit not in orbits:
            orbits[orbit] = [item]
        else:
            orbits[orbit].append(item)
    return orbits

In [13]:
def export_tif(bands, profile, export_path):
    array = np.array(bands)
    profile.update(count=array.shape[0])
    with rio.open(export_path, "w", **profile) as dst:
        dst.write(array)

In [14]:
s2_grid = s2_grid.sample(frac=1)

In [15]:
s2_grid.head()

,Name,geometry
19,55GFR,"MULTIPOLYGON Z (((148.16715 -39.74386 0.00000,..."
2,54HYB,"MULTIPOLYGON Z (((143.30345 -38.82574 0.00000,..."
22,55HDS,"MULTIPOLYGON Z (((145.84726 -38.84277 0.00000,..."
11,55GDQ,"MULTIPOLYGON Z (((145.81683 -40.64479 0.00000,..."
4,55GBR,"MULTIPOLYGON Z (((143.50105 -39.69697 0.00000,..."


In [16]:
target_bands = 12
time_steps = 6

In [ ]:
for id, row in s2_grid.iterrows():
    centroid = row.geometry.centroid
    export_path = export_dir / f"{row.Name}_{extract_start_year}_{extract_end_year}.tif"
    print(export_path)

    if export_path.exists():
        print(f"File exists for {row.Name}")
        continue

    # Sentinel-2 query parameters
    query = {
        "collections": ["sentinel-2-l2a"],
        "intersects": shapely.to_geojson(centroid),
        "datetime": f"{extract_start_year}-01-01T00:00:00Z/{extract_end_year}-12-31T23:59:59Z",
    }
    scenes = catalog.search(**query).get_all_items()
    # break
    if len(scenes) == 0:
        continue

    scenes_by_orbit = split_by_orbits(scenes)
    all_orbits_bands = []
    for orbit, scenes in scenes_by_orbit.items():
        items_filtered = sort_by_clouds(row, scenes)
        items_filtered = items_filtered[:20]
        # break
        items_with_tide = filter_by_tide(centroid, items_filtered, limit=8)
        bands, profile = downlaod_bands(items_with_tide, time_steps)
        all_orbits_bands.append(bands)
        # break
    # break

    all_orbits_bands = np.array(all_orbits_bands)
    all_orbits_bands = np.moveaxis(all_orbits_bands, 0, 1)

    merged_bands = []
    for multi_orbit_bands in all_orbits_bands:
        target_array = np.zeros(multi_orbit_bands.shape[1:])
        for band in multi_orbit_bands:
            target_array[target_array == 0] = band[target_array == 0]
        merged_bands.append(target_array)
    # merged_bands = np.array(merged_bands)

    if len(merged_bands) == target_bands:
        export_tif(merged_bands, profile, export_path)
    else:
        print(f"Failed to download {row.Name}")
    # break

In [17]:
scenes = list(export_dir.glob("*.tif"))
len(scenes)

24

In [18]:
# scenes = [Path("/Volumes/4TB SSD/Coastline data/small test.tif")]

In [21]:
for scene in scenes:
    stack_src = rio.open(scene)
    local_bounds = shapely.geometry.box(*stack_src.bounds)

    original_crs = stack_src.profile["crs"]
    target_crs = pyproj.CRS("EPSG:4326")
    # Create a pyproj transformer
    project = pyproj.Transformer.from_crs(
        original_crs, target_crs, always_xy=True
    ).transform

    # Transform the polygon
    wgs_bounds = transform(project, local_bounds)

    search = catalog.search(
        collections=["sentinel-1-rtc"],
        intersects=wgs_bounds,
    )
    items = search.item_collection()
    srcs = []
    nodata_count = True
    merge_list = []
    parts = []
    part_number = 0
    for item in items:
        href = item.assets["vh"].href
        src = rio.open(planetary_computer.sign(href))
        if src.crs != original_crs:
            continue

        window = rio.windows.from_bounds(*stack_src.bounds, src.transform)
        array = src.read(1, window=window)
        transfm = rio.windows.transform(window, transform=src.transform)
        profile = src.profile.copy()

        profile.update(
            width=array.shape[-1],
            height=array.shape[-2],
            transform=transfm,
            count=1,
            dtype="float32",
        )
        sar_path = Path(str(scene).replace(".tif", f"_{part_number}_sar.tif"))
        part_number += 1
        parts.append(sar_path)
        with rio.open(sar_path, "w", **profile) as dst:
            dst.write(array, 1)

        for i in parts:
            merge_list.append(rio.open(sar_path))

        mosaic, out_trans = merge(merge_list)
        # count zeros in array
        nodata_count = np.count_nonzero(mosaic == -32768)
        print(nodata_count)
        if nodata_count == 0:
            break
    sar_path = Path(str(scene).replace(".tif", f"_sar.tif"))

    profile.update(
        width=mosaic.shape[-1],
        height=mosaic.shape[-2],
        transform=out_trans,
        count=1,
        dtype="float32",
    )
    with rio.open(sar_path, "w", **profile) as dst:
        dst.write(mosaic)

49038440
35657766
855
851
0
56829186
60649
3763
3763
3654
3654
3654
3654
3654


In [32]:
sar_path

PosixPath('/Volumes/4TB SSD/Coastline data/small test_0_sar.tif')

In [28]:
profile.update(count=1, dtype="float32")

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -32768.0, 'width': 1163.0, 'height': 1140.0, 'count': 1, 'crs': CRS.from_epsg(32755), 'transform': Affine(10.0, 0.0, 524080.0,
       0.0, -10.0, 5470450.0), 'blockxsize': 512, 'blockysize': 512, 'tiled': True, 'compress': 'deflate', 'interleave': 'band'}

In [ ]:
local_bounds.bounds

In [ ]:
planetary_computer.sign(href)

In [ ]:
profile = stack_src.profile.copy()
profile["count"] = 1
profile["dtype"] = "float32"

with rio.open("test.tif", "w", **profile) as dst:
    dst.write(array, 1)

In [ ]:
array.shape

In [ ]:
scene